In [ ]:
!pip install dwave-ocean-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 385.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.6/305.6 kB 9.9 MB/s eta 0:00:00


In [ ]:
import networkx as nx
from dimod import BinaryQuadraticModel
import numpy as np

In [ ]:
def generate_qubo(graph, k, A, B):
    n = len(graph)
    Q = np.zeros((n, n))

    # Penalty for selecting exactly k vertices
    for i in range(n):
        Q[i, i] += A - A * (2 * k)
        for j in range(i + 1, n):
            Q[i, j] += 2*A

    # Penalty for selecting non-adjacent vertices
    for i in range(n):
        for j in range(i + 1, n):
            if graph[i][j] != 0:  # If there's no edge between i and j
                Q[i, j] += -2*B # * k *(k-1)

    return Q


In [ ]:
# Example graph adjacency matrix
graph = [
    [0, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [0, 1, 1, 0]
]

# Size of the clique to find
k = 3
A = 5
B = 1
# Generate QUBO matrix
qubo_matrix = generate_qubo(graph, k, A, B)
print(qubo_matrix)
# Convert QUBO matrix to dictionary format

Q1 = {(i, j): qubo_matrix[i][j] for i in range(len(graph)) for j in range(len(graph)) if (qubo_matrix[i][j] != 0 and i != j)}
Q2 = {(i): qubo_matrix[i][j] for i in range(len(graph)) for j in range(len(graph)) if (qubo_matrix[i][j] != 0 and i == j)}
print(Q1)
print(Q2)
bqm = BinaryQuadraticModel(Q2, Q1, 0.0, 'BINARY')
print(bqm)

[[-25.   8.   8.  10.]
 [  0. -25.   8.   8.]
 [  0.   0. -25.   8.]
 [  0.   0.   0. -25.]]
{(0, 1): 8.0, (0, 2): 8.0, (0, 3): 10.0, (1, 2): 8.0, (1, 3): 8.0, (2, 3): 8.0}
{0: -25.0, 1: -25.0, 2: -25.0, 3: -25.0}
BinaryQuadraticModel({0: -25.0, 1: -25.0, 2: -25.0, 3: -25.0}, {(1, 0): 8.0, (2, 0): 8.0, (2, 1): 8.0, (3, 0): 10.0, (3, 1): 8.0, (3, 2): 8.0}, 0.0, 'BINARY')


In [ ]:
#Using Dwave Quantum Computer to solve the QUBO problem
token = 'DEV-abdc4c7339474eacea7a99b9e4269e22f667d3f2'
solver = 'Advantage_system6.4'

In [ ]:
from dimod import BinaryQuadraticModel
from dwave.system import EmbeddingComposite, DWaveSampler

In [ ]:
sampler = EmbeddingComposite(DWaveSampler(token= token, solver = solver))
sampleset = sampler.sample(bqm, num_reads=100)
# Get the best solution

print(sampleset)
solution = sampleset.first.sample
energy = sampleset.first.energy
print(energy)

   0  1  2  3 energy num_oc. chain_.
0  1  1  1  0  -51.0      34     0.0
1  0  1  1  1  -51.0      60     0.0
2  1  1  1  1  -50.0       5     0.0
3  1  1  0  1  -49.0       1     0.0
['BINARY', 4 rows, 100 samples, 4 variables]
-51.0
